In [33]:
%pip install faker



   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------------- - 1.8/1.9 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 13.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Data Preprocess


## Import Libraries


In [1]:
import pandas as pd


## Initialize Dataset


In [2]:
file_path = '../dataset/dataset.csv'


In [ ]:

# Load dataset from CSV file
df = pd.read_csv(file_path)

df = df.drop(columns=['flags'])

# Display the first few rows of the dataframe
df.head()


,instruction,category,intent,response
0,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [6]:
# Check the total length of the dataframe
total_length = len(df)
print(f"Total length of the dataframe: {total_length}")

# Check for any missing or empty values in each column
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Check for any missing or empty values in each row
missing_values_rows = df.isnull().sum(axis=1)
print("Missing values in each row:")
print(missing_values_rows)


Total length of the dataframe: 26872
Missing values in each column:
instruction    0
category       0
intent         0
response       0
dtype: int64
Missing values in each row:
0        0
1        0
2        0
3        0
4        0
        ..
26867    0
26868    0
26869    0
26870    0
26871    0
Length: 26872, dtype: int64


## Checking of Dataset


In [4]:
unique_intents = df['intent'].unique()
unique_categories = df['category'].unique()

print("Unique intents:")
print(unique_intents)
print("Total unique intents:", len(unique_intents))

print("\nUnique categories:")
print(unique_categories)
print("Total unique categories:", len(unique_categories))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']
Total unique intents: 27

Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']
Total unique categories: 11


## Category and Intent


Intent


In [18]:
# Get the unique intents
unique_intents_list = df['intent'].unique()

# Print the unique intents
print("Unique intents:")
print(unique_intents_list)

# Get the distribution of intents
intent_distribution = df['intent'].value_counts()

# Print the distribution of intents
print("\nIntent distribution:")
print(intent_distribution)

print("Total unique intents:", len(unique_intents_list))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']

Intent distribution:
intent
edit_account                1000
switch_account              1000
check_invoice               1000
complaint                   1000
contact_customer_service    1000
delivery_period              999
registration_problems        999
check_payment_methods        999
contact_human_agent          999
payment_issue                999
newsletter_subscription      999
get_invoice                  999
place_order                  998
cancel_order        

Category


In [31]:
# Get the unique categories
unique_categories_list = df['category'].unique()

# Print the unique categories
print("Unique categories:")
print(unique_categories_list)

# Get the distribution of categories
category_distribution = df['category'].value_counts()

# Print the distribution of categories
print("\nCategory distribution:")
print(category_distribution)

print("Total unique categories:", len(unique_categories_list))


Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']

Category distribution:
category
ACCOUNT         5986
ORDER           3988
REFUND          2992
INVOICE         1999
CONTACT         1999
PAYMENT         1998
FEEDBACK        1997
DELIVERY        1994
SHIPPING        1970
SUBSCRIPTION     999
CANCEL           950
Name: count, dtype: int64
Total unique categories: 11


## Entity


### Placeholder


Checking placeholder


In [34]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders in the 'instruction' column
df['has_placeholder_instruction'] = df['instruction'].str.contains(r'\{\{.*?\}\}')
df['has_placeholder'] = df['has_placeholder_instruction']

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Extract placeholders
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', text)

# Apply the function to rows with placeholders and get all placeholders from the 'instruction' column
placeholders_instruction = df[df['has_placeholder_instruction']]['instruction'].apply(extract_placeholders).explode()

# Count the occurrences of each placeholder
placeholder_counts = Counter(placeholders_instruction)

# Convert the counts to a DataFrame for better visualization
placeholder_df = pd.DataFrame.from_dict(placeholder_counts, orient='index', columns=['count'])


# Print the total rows with placeholders
print(placeholder_df)

# Print the total rows of the dataset and check for any missing values
print("Total rows in the dataset:", len(df))


                  count
Order Number       2907
Invoice Number        8
Person Name         887
Account Type       1011
Account Category    822
Delivery City       234
Delivery Country    177
Currency Symbol     372
Refund Amount       624
Total rows in the dataset: 26872


In [35]:

from faker import Faker
import random

fake = Faker()

def replace_placeholders(text):
    replacements = {
        "{{Order Number}}": str(random.randint(10000, 99999)),
        "{{Person Name}}": fake.name(),
        "{{Account Type}}": fake.random_element(elements=("Premium", "Basic", "Business")),
        "{{Account Category}}": fake.random_element(elements=("Personal", "Corporate")),
        "{{Delivery City}}": fake.city(),
        "{{Delivery Country}}": fake.country(),
        "{{Currency Symbol}}": fake.currency_symbol(),
        "{{Refund Amount}}": f"${random.randint(10, 1000)}",
    }
    for placeholder, value in replacements.items():
        text = text.replace(placeholder, value)
    return text

df['instruction'] = df['instruction'].apply(replace_placeholders)


In [36]:
placeholder_mapping = {
    "{{Delivery City}}": "Location",
    "{{Delivery Country}}": "Location",
}

def map_placeholders_to_groups(text, mapping):
    for placeholder, group in mapping.items():
        text = text.replace(placeholder, group)
    return text

df['instruction'] = df['instruction'].apply(lambda x: map_placeholders_to_groups(x, placeholder_mapping))


In [37]:
def remove_rare_placeholders(text, rare_placeholders):
    for placeholder in rare_placeholders:
        text = text.replace(f"{{{{{placeholder}}}}}", "")
    return text

rare_placeholders = ["Invoice Number"]  # Add other rare placeholders if needed
df['instruction'] = df['instruction'].apply(lambda x: remove_rare_placeholders(x, rare_placeholders))


In [39]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set the padding token

# Function to tokenize text
def tokenize_text(text):
    return tokenizer.encode(text, truncation=True, padding="max_length", max_length=512)

df['tokenized_instruction'] = df['instruction'].apply(tokenize_text)
df['tokenized_response'] = df['response'].apply(tokenize_text)


In [41]:
# Ensure the 'entities' column exists in the dataframe
if 'entities' not in df.columns:
    df['entities'] = [[] for _ in range(len(df))]

def align_labels_with_tokens(text, entities):
    tokens = tokenizer.tokenize(text)
    labels = ["O"] * len(tokens)  # Initialize all labels as "O"
    
    for entity in entities:
        start, end, label = entity
        token_start = None
        token_end = None
        
        # Find token indices corresponding to character positions
        for i, token in enumerate(tokens):
            if tokenizer.convert_tokens_to_string(token).startswith(text[start:end]):
                token_start = i
                token_end = i + len(tokenizer.tokenize(text[start:end]))
                break
        
        if token_start is not None and token_end is not None:
            labels[token_start] = f"B-{label}"  # Begin tag
            for i in range(token_start + 1, token_end):
                labels[i] = f"I-{label}"  # Inside tag
    
    return labels

# Example: Add aligned labels for NER
df['aligned_labels'] = df.apply(
    lambda row: align_labels_with_tokens(row['instruction'], row['entities']), axis=1
)

df.to_csv('../dataset/dataset_preprocessed.csv', index=False)
